In [1]:
import sys

PWD = '../../..'
sys.path.append(PWD)

import numpy as np
import pandas as pd

import tensorflow as tf

tf.get_logger().setLevel('ERROR')
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
import keras_tuner as kt

from src.network_utils import count_params
from src.cross_validator import KerasTunerCrossValidator
from src.dataset import load_dataset_train_test, load_dataset_train_val
from src.models import hp_unet_builder as hp_model_builder

In [2]:
DATASET_PATH = PWD + '/data/dataset/dataset.pkl'
TRIALS_DIR = PWD + '/data/tuner'
CROSSVAL_DIR = PWD + '/data/cross_val'

PLANE = 2
CHANNEL = 11

PROJECT_NAME = f'unet_{PLANE}_{CHANNEL}'

LR = 0.01

N_EPOCHS = 3000
BATCH_SIZE = 2048
MAX_TRIALS = 40
EXECUTIONS_PER_TRIAL = 2

TOP_N = 4
CROSSVAL_N_CV = 5
CROSSVAL_N_EXEC = 2
LOSS_WEIGHT = 1000

In [3]:
X_base_train, _, y_base_train_original, _ = load_dataset_train_test(PWD, PLANE, CHANNEL)
X_train, X_val, y_train_original, y_val_original = load_dataset_train_val(PWD, PLANE, CHANNEL)

X_base_train.shape, X_train.shape, X_val.shape

((22412, 24), (17929, 24), (4483, 24))

In [4]:
def gaussian_kernel(mu, sigma=1., n=24):
    x = np.arange(0, n)
    return np.exp(-(x - mu) ** 2 / (2 * sigma ** 2))


y_base_train = np.array([gaussian_kernel(y) for y in y_base_train_original])
y_train = np.array([gaussian_kernel(y) for y in y_train_original])
y_val = np.array([gaussian_kernel(y) for y in y_val_original])

y_base_train.shape, y_train.shape, y_val.shape

((22412, 24), (17929, 24), (4483, 24))

# Model

In [5]:
def model_builder(hp: kt.HyperParameters) -> keras.Model:
    model = hp_model_builder(hp)
    model.compile(loss='mse', optimizer=optimizers.Adam(LR), loss_weights=LOSS_WEIGHT)
    return model

In [6]:
model_builder(kt.HyperParameters()).summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 24)]         0           []                               
                                                                                                  
 reshape (Reshape)              (None, 24, 1)        0           ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 24, 32)       96          ['reshape[0][0]']                
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 12, 32)       0           ['conv1d[0][0]']                 
                                                                                              

In [7]:
model_callbacks = [
    callbacks.EarlyStopping(patience=50),
    callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=10),
    callbacks.TensorBoard("/tmp/tb_logs/bayesian/" + PROJECT_NAME)
]

# Bayesian tuner

In [8]:
bayesian_tuner = kt.BayesianOptimization(model_builder, objective='val_loss', executions_per_trial=EXECUTIONS_PER_TRIAL,
                                         max_trials=MAX_TRIALS, directory=TRIALS_DIR + "/bayesian",
                                         project_name=PROJECT_NAME, overwrite=False)

bayesian_tuner.search(X_train, y_train, validation_data=[X_val, y_val], epochs=N_EPOCHS, callbacks=model_callbacks,
                      batch_size=BATCH_SIZE, verbose=3)

# Results

In [9]:
bayesian_tuner.results_summary(1)

Results summary
Results in ../../../data/tuner/bayesian\unet_2_11
Showing 1 best trials
Trial summary
Hyperparameters:
unet_depth: 3
n_conv_layers: 3
conv_filters_mult: 1
conv_spatial_dropout: 0.0
batch_normalization: False
input_batch_normalization: True
Score: 0.310469850897789


In [10]:
best_model = bayesian_tuner.get_best_models(num_models=1)[0]
best_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 24)]         0           []                               
                                                                                                  
 reshape (Reshape)              (None, 24, 1)        0           ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 24, 1)       4           ['reshape[0][0]']                
 alization)                                                                                       
                                                                                                  
 conv1d (Conv1D)                (None, 24, 8)        24          ['batch_normalization[0][0]']

---

In [11]:
for i, hyperparameters in enumerate(bayesian_tuner.get_best_hyperparameters(TOP_N)):
    print(f'========== Model {i} ==========')
    print(hyperparameters.get_config()['values'])
    model_tmp = model_builder(hyperparameters)
    print('Number of parameters:', count_params(model_tmp))

========== Model 0 ==========
{'unet_depth': 3, 'n_conv_layers': 3, 'conv_filters_mult': 1, 'conv_spatial_dropout': 0.0, 'batch_normalization': False, 'input_batch_normalization': True}
Number of parameters: 30051
========== Model 1 ==========
{'unet_depth': 2, 'n_conv_layers': 3, 'conv_filters_mult': 1, 'conv_spatial_dropout': 0.0, 'batch_normalization': False, 'input_batch_normalization': False}
Number of parameters: 7265
========== Model 2 ==========
{'unet_depth': 2, 'n_conv_layers': 3, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.0, 'batch_normalization': False, 'input_batch_normalization': True}
Number of parameters: 113539
========== Model 3 ==========
{'unet_depth': 3, 'n_conv_layers': 1, 'conv_filters_mult': 1, 'conv_spatial_dropout': 0.0, 'batch_normalization': False, 'input_batch_normalization': False}
Number of parameters: 16385


In [12]:
bayesian_tuner.results_summary(TOP_N)

Results summary
Results in ../../../data/tuner/bayesian\unet_2_11
Showing 4 best trials
Trial summary
Hyperparameters:
unet_depth: 3
n_conv_layers: 3
conv_filters_mult: 1
conv_spatial_dropout: 0.0
batch_normalization: False
input_batch_normalization: True
Score: 0.310469850897789
Trial summary
Hyperparameters:
unet_depth: 2
n_conv_layers: 3
conv_filters_mult: 1
conv_spatial_dropout: 0.0
batch_normalization: False
input_batch_normalization: False
Score: 0.3137176036834717
Trial summary
Hyperparameters:
unet_depth: 2
n_conv_layers: 3
conv_filters_mult: 4
conv_spatial_dropout: 0.0
batch_normalization: False
input_batch_normalization: True
Score: 0.3140224814414978
Trial summary
Hyperparameters:
unet_depth: 3
n_conv_layers: 1
conv_filters_mult: 1
conv_spatial_dropout: 0.0
batch_normalization: False
input_batch_normalization: False
Score: 0.315817266702652


# Cross-validation for top 5 models

In [13]:
cross_validator = KerasTunerCrossValidator(bayesian_tuner, X_base_train, y_base_train, model_builder,
                                           directory=CROSSVAL_DIR, project_name=PROJECT_NAME,
                                           n_epochs=N_EPOCHS, batch_size=BATCH_SIZE, n_top=TOP_N,
                                           n_cv=CROSSVAL_N_CV, n_executions=CROSSVAL_N_EXEC, overwrite=False)
model_scores = cross_validator()

{'unet_depth': 3, 'n_conv_layers': 3, 'conv_filters_mult': 1, 'conv_spatial_dropout': 0.0, 'batch_normalization': False, 'input_batch_normalization': True}
Number of parameters: 30051
Got score: 0.3133 (0.3138, 0.3127)
Got score: 0.2994 (0.3000, 0.2988)
Got score: 0.2929 (0.2923, 0.2936)
Got score: 0.3056 (0.3034, 0.3077)
Got score: 0.3148 (0.3148, 0.3149)


{'unet_depth': 2, 'n_conv_layers': 3, 'conv_filters_mult': 1, 'conv_spatial_dropout': 0.0, 'batch_normalization': False, 'input_batch_normalization': False}
Number of parameters: 7265
Got score: 0.5751 (0.8355, 0.3146)
Got score: 0.3023 (0.3036, 0.3010)
Got score: 0.5472 (0.8021, 0.2923)
Got score: 0.3060 (0.3035, 0.3085)
Got score: 0.3219 (0.3234, 0.3203)


{'unet_depth': 2, 'n_conv_layers': 3, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.0, 'batch_normalization': False, 'input_batch_normalization': True}
Number of parameters: 113539
Got score: 0.3179 (0.3228, 0.3129)
Got score: 0.3004 (0.2990, 0.3017)
Got score: 0.2961 (0.3022, 0.2900)
Got score: 0.3030 (0.3048, 0.3011)
Got score: 0.3183 (0.3191, 0.3176)


{'unet_depth': 3, 'n_conv_layers': 1, 'conv_filters_mult': 1, 'conv_spatial_dropout': 0.0, 'batch_normalization': False, 'input_batch_normalization': False}
Number of parameters: 16385
Got score: 0.3121 (0.3117, 0.3125)
Got score: 0.3030 (0.3045, 0.3015)
Got score: 0.2941 (0.2934, 0.2948)
Got score: 0.3068 (0.3049, 0.3087)
Got score: 0.3205 (0.3223, 0.3187)


In [14]:
mean_scores = [f"{np.mean(scores):0.3f}" for scores in model_scores.values()]
std_scores = [f"{np.std(scores):0.3f}" for scores in model_scores.values()]
n_params = [count_params(model_builder(hyperparameters)) for hyperparameters in
            bayesian_tuner.get_best_hyperparameters(TOP_N)]

df = pd.DataFrame({'mean': mean_scores, 'std': std_scores, 'n_params': n_params}, index=model_scores.keys())
df.index.name = 'Model'
df

,mean,std,n_params
Model,,,
0,0.305,0.008,30051
1,0.410,0.124,7265
2,0.307,0.009,113539
3,0.307,0.009,16385
